# Lesson #02 - The beta-binomial model

This notebook explores the **beta-binomial model**. It is strongly based on Ref. *Machine Learning : A Probabilistic Perspective*, by Kevin P. Murphy, 2012, ISBN 978-0-262-01802-9, pp. 72-78.

It was written by Bruno Abreu: https://sites.google.com/view/brunoabreu/home

Last time this notebook was updated was on: 07-23-2020.

Please, feel free to explore it. The author appreciates being credited.

## Introduction

In the last lesson, we have constructed a framework to infer a distribution over a discrete variable that is drawn from an instance of a finite hypothesis space, $h \in H$, given a series of discrete observations $D$.

Here, we will analyze a similar situation, where now the unknown parameters are continuous, so the hypothesis space is some subset of $\mathbb{R}^K$, where $K$ is the number of parameters.

For that, consider the problem of inferring the probability that a coin shows up heads, given a series of observed coin tosses. Although this might look simple, it was the example analyzed by Bayes in his original paper of 1763.
We will proceed exactly like before: specify priors and likelihood, and then deriving posterior and posterior predictive distributions.

## Likelihood

Suppose that $X_i \sim Ber(\theta)$, where $Ber(p)$ is the Bernoullu distribution, the discrete probability distribution of a random variable that takes the value 1 with probability $p$ and the value 0 with probability $q = 1- p$. It can be thought of as a model for the set of possible outcomes of any single experiment that asks a yes-no question. 

In our coin-toss example, $X_i = 1$ represents “heads”, $X_i = 0$ represents “tails”, and $\theta \in [0,1]$ is the rate parameter (probability of heads). In case the data is **independently and identically distributed** (iid), the likelihood has the form

$p(D|\theta) = \theta^{N_1} (1 – \theta)^{N_0},$

where we have $N_1 = \sum_{i=1}^N \mathbb{I}(x_i=1)$ heads and $N_0 =\sum_{i=1}^N \mathbb{I}(x_i=0)$ tails. These two counts are called **sufficient statistics** of the data because they are all we need to know about $D$ to infer $\theta$.

Now suppose that the data consists of the count of the number of heads $N_1$ observed in a fixed number $N = N_1 + N_0$ of trials. In this case, we would have $N_1 \sim Bin(N,\theta)$, where $Bin$ is the binomial distribution,

$Bin(k | n,\theta) = {n\choose k}\theta^k(1-\theta)^{n-k}.$

Since $n\choose k$ is a constant that does not depend on $\theta$, the likelihood for the binomial sampling model is the same as the likelihood for the Bernoulli model. Therefore, any infereces that we make about $\theta$ will be the same whether we observe the counts, $D = (N_1, N)$, or a sequence of trials, $D = \{x_1, …, x_N\}$.

## Prior

We need a prior that has support over the interval $[0,1]$. Notice that a convenient one would be

$p(\theta) \propto \theta^{\gamma_1} (1-\theta)^{\gamma_2}$,

for some prior parameters $\gamma_1$ and $\gamma_2$, simply because the posterior would be easily written as

$p(\theta|D) \propto p(D|\theta)p(\theta) = \theta^{N_1+\gamma_1}(1-\theta)^{N_0+\gamma_2}$.

When prior and posterior have the same form, the prior is called a **conjugate prior** for the corresponding likelihood. They usually simplify computation.

In the case of the Bernoulli distribution, the conjugate prior is the beta distribution:

$ Beta(\theta | a,b) \propto \theta^{a-1}(1-\theta)^{b-1}$.

This distribution can be used to model events that are constrained to take place within an interval defined by a minimum and a maximum value, such as to describe the time to completion and the cost of a task in project management. In a more general context, it is used to model the behavior of random variables limited to intervals of finite length. 

The two parameters here are called **hyper-parameters**, and they can be set in order to encode our prior beliefs, like we did before. For instance, to encode our beliefs that $\theta$ has mean 0.7 and standard deviation 0.2, we would set $a = 2.975$ and $b = 1.275$. If we know nothing about $\theta$, except that it is in the interval $[0,1]$, we can use a uniform prior, with $a = b = 1$.

## Posterior

If we multiply the likelihood by the beta prior, we get

$p(\theta |D) \propto Bin(N_1|\theta, N_0+N_1) Beta(\theta|a,b) \propto Beta(\theta|N_1+a, N_0+b)$,

which is just adding the prior hyper-parameters to the empirical counts. For this reason, they are also called **pseudo counts**. The strength of the prior, sometimes called the **effective sample size** of the prior, is the sum of its pseudo counts. 

Since this posterior is a well-known distribution, we can easily calculate the statistical properties of this model. For instance, check the material about the Beta distribution here: https://en.wikipedia.org/wiki/Beta_distribution

### Mean and mode
The MAP estimate will be given by

$ \hat{\theta}_{MAP} = \frac{a + N_1 - 1}{a + b + N - 2}$.

By using a uniform prior, the MAP estimate reduces to the MLE, which is, of course, just the empirical fraction of heads:

$\hat{\theta}_{MLE} = N_1 / N$.

By contrast, the posterior mean is given by

$ \bar{\theta} = \frac{a+N_1}{a+b+N} $.

It is possible to explicitly verify the notion that the posterior is some sort of compromise between what we previously believed (prior) and what the data is telling us (likelihood).

Let $\alpha_0 = a + b$ be the equivalent sample size of the prior, and the prior mean be $m_1 = a/\alpha_0$. Then, the posterior mean is

$ \mathbb{E}[\theta|D] = \frac{\alpha_0 m_1 + N}{N + alpha_0} = \frac{\alpha_0}{N + \alpha_0}m_1 + \frac{N}{N + \alpha_0}\frac{N_1}{N} = \lambda m_1 + (1-\lambda)\hat{\theta}_{MLE}$,

where $\lambda = \alpha_0 / (N+\alpha_0)$ is the ratio of the prior to posterior equivalent sample size. Therefore, the posterior mean is a convex combination of the prior mean and MLE.

### Variance
In order to know how much we can trust these estimates, we can measure the variance of the posterior. It is given by

$ var[\theta|D] = \frac{(a+N_1)(b+N_0)}{(a+N_1+b+N_0)^2(a+N_1+b+N_0+1)}, $

which can be simplified in the limite $N \gg a,b$, to

$ var[\theta|D] \approx \frac{N_1 N_0}{N^3} = \frac{\hat{\theta}(1-\hat{\theta})}{N}$,

where $\hat{\theta}$ is the MLE. Therefore, the **error bar** of the estimate is given by

$ \sigma = \sqrt{var[\theta|D]} \approx \sqrt{\frac{\hat{\theta}(1-\hat{\theta})}{N}}$. 

Notice that it goes down with $N$, and also that it is maximized when $\hat{\theta}=0.5$ and minimized when $\hat{\theta}$ is close to either 0 or 1. In other words, it is easier to be sure that a coin is biased than to be sure that it is fair.

## Posterior predictive distribution

# Cooking things down to code

As you may notice, a lot of things in this lesson are actually analytical, which is different from the previous one where we had quite a bit to code. However, it is interesting to practice our ability to explore Python libraries that provide support to different probability distributions, as they lie within the heart of whatever model we come across. 